In [1]:
from datetime import datetime, timedelta
import urllib.request

## For inference data from Visual Crossing we handle it using their free API.

In [2]:
date_format = "%Y-%m-%d"
end_date = (datetime.now()- timedelta(days=1)).strftime(date_format)
api_key = 'UXNEEH5UKH8YXK94KJY3HMWF7'
url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Central%20Park%20New%20York%20City/2023-09-22/{end_date}?unitGroup=us&include=days&key={api_key}&contentType=csv'
print(url)
destination = f'Data/infer_temp.csv' # this becomes out inference data
urllib.request.urlretrieve(url, destination)


https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Central%20Park%20New%20York%20City/2023-09-22/2023-10-11?unitGroup=us&include=days&key=UXNEEH5UKH8YXK94KJY3HMWF7&contentType=csv


('Data/infer_temp.csv', <http.client.HTTPMessage at 0x105626af0>)

In [3]:
import pandas as pd
import numpy as np
import pickle

In [4]:
df = pd.read_csv(destination)
# df.head()

In [11]:
df.tail() # latest data upto yesterday


,name,DATE,TMAX,TMIN,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
15,"Central Park, New York, NY 10021, United States",2023-10-07,70.1,54.1,64.1,70.1,54.1,64.1,59.4,85.2,...,5.0,3,10,2023-10-07T06:58:23,2023-10-07T18:28:34,0.77,"Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
16,"Central Park, New York, NY 10021, United States",2023-10-08,57.8,49.9,54.6,57.8,45.8,54.5,42.9,65.0,...,11.6,8,10,2023-10-08T06:59:26,2023-10-08T18:26:57,0.80,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KTEB,KLGA,F1417,KNYC"
17,"Central Park, New York, NY 10021, United States",2023-10-09,58.9,48.8,54.1,58.9,44.5,53.5,38.8,56.7,...,13.8,9,10,2023-10-09T07:00:29,2023-10-09T18:25:21,0.83,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KTEB,KLGA,F1417,KNYC"
18,"Central Park, New York, NY 10021, United States",2023-10-10,62.2,53.2,57.4,62.2,53.2,57.4,45.7,65.6,...,12.7,7,10,2023-10-10T07:01:32,2023-10-10T18:23:45,0.87,"Rain, Partially cloudy",Partly cloudy throughout the day with early mo...,rain,"D9152,KTEB,KLGA,F1417,KNYC"
19,"Central Park, New York, NY 10021, United States",2023-10-11,69.2,55.1,61.2,69.2,55.1,61.2,46.6,60.9,...,16.6,7,10,2023-10-11T07:02:36,2023-10-11T18:22:10,0.90,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"D9152,KTEB,KLGA,KNYC,F1417"


In [12]:
file = open("models/lin_reg.pkl",'rb')
model = pickle.load(file)
file.close()

In [13]:
def give_next24hrs_pred(model, df):

    df.rename(columns = {'datetime':'DATE', 'tempmax':'TMAX', 'tempmin':'TMIN'}, inplace = True)

    df = df[['DATE', 'TMAX', 'TMIN']]

    df['TMAX_prev7'] = df['TMAX'].rolling(7).max()
    df['TMAX_prev3'] = df['TMAX'].rolling(3).max()
    df['TMAX_prev2'] = df['TMAX'].rolling(2).max()

    df['TMAX_prev1'] = df['TMAX'].rolling(1).max()

    df['TMIN_prev3'] = df['TMIN'].rolling(3).max()
    df['TMIN_prev2'] = df['TMIN'].rolling(2).max()
    df['TMIN_prev1'] = df['TMIN'].rolling(1).max()

    df = df.loc[7:, :]

    df['DATE'] = pd.to_datetime(df['DATE'])

    df['day'] = df['DATE'].dt.day
    df['month'] = df['DATE'].dt.month

    df.sort_values(by = ['DATE'], inplace = True)
    df.set_index("DATE", inplace = True)

    df.fillna(-1, inplace = True)

    df.drop(['TMIN', 'TMAX'], axis = 1, inplace = True)

    print(df)

    return model.predict(df)




In [14]:
preds = give_next24hrs_pred(model, df)

            TMAX_prev7  TMAX_prev3  TMAX_prev2  TMAX_prev1  TMIN_prev3  \
DATE                                                                     
2023-09-29        65.9        64.1        64.1        63.0        60.2   
2023-09-30        65.9        65.9        65.9        65.9        60.2   
2023-10-01        75.8        75.8        75.8        75.8        60.2   
2023-10-02        76.9        76.9        76.9        76.9        63.0   
2023-10-03        80.1        80.1        80.1        80.1        63.0   
2023-10-04        82.1        82.1        82.1        82.1        65.9   
2023-10-05        82.1        82.1        82.1        75.8        65.9   
2023-10-06        82.1        82.1        75.8        71.1        66.8   
2023-10-07        82.1        75.8        71.1        70.1        66.8   
2023-10-08        82.1        71.1        70.1        57.8        66.8   
2023-10-09        82.1        70.1        58.9        58.9        54.1   
2023-10-10        82.1        62.2    

/var/folders/_p/1tsg949n0qzgrfk4rclgt1k80000gn/T/ipykernel_20190/509194500.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TMAX_prev7'] = df['TMAX'].rolling(7).max()
/var/folders/_p/1tsg949n0qzgrfk4rclgt1k80000gn/T/ipykernel_20190/509194500.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TMAX_prev3'] = df['TMAX'].rolling(3).max()
/var/folders/_p/1tsg949n0qzgrfk4rclgt1k80000gn/T/ipykernel_20190/509194500.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [17]:
print('Prediction next day temperature : ', preds[-1])

Prediction next day temperature :  68.87018
